# Wikipics async demo

## Referências sobre `asyncio` no Jupyter Notebook

* [IPython 7.0, Async REPL (from 2018)](https://blog.jupyter.org/ipython-7-0-async-repl-a35ce050f7f7)
* [Supporting asyncio.get_event_loop().run_until_complete() in repls](https://discuss.python.org/t/supporting-asyncio-get-event-loop-run-until-complete-in-repls/5573/9)
* [Running Asynchronous Code in Jupyter Notebooks: Managing Event Loops](https://medium.com/p/b9696a596ce4) (paywall, não vale o custo)

In [1]:
from pathlib import Path
from typing import NamedTuple

from IPython.display import display
from ipywidgets.widgets import Image, Layout
import httpx

from wikipics import get_sample_url, get_sample_urls

class ImageRecord(NamedTuple):
    pixels: bytes
    name: str
    size: int

Função que usamos na demonstração com threads e processos, denominada `fetch` em 
[wikipics-demo](wikipics-demo.ipynb).
Reproduzi aqui para comparar com a versão assíncrona de `fetch`, logo abaixo.

```python
def fetch_blocking(url) -> ImageRecord:
    resp = httpx.get(url)  # thread blocked here
    resp.raise_for_status()
    name = Path(url).name
    return ImageRecord(resp.content, name, len(resp.content))
```

In [3]:
from httpx import AsyncClient

async def fetch(client:AsyncClient, url:str) -> ImageRecord:
    resp = await client.get(url)
    resp.raise_for_status()
    name = Path(url).name
    return ImageRecord(resp.content, name, len(resp.content))

Outro trecho de código de [wikipics-demo](wikipics-demo.ipynb),
para comparação com o código usando `await` em seguida.

```python
url = get_sample_url(1_000_000)
img_rec = fetch_blocking(url)  # main thread blocks here
print(f'{img_rec.size:12_} bytes | {img_rec.name}')
Image(value=img_rec.pixels)
```

In [4]:
url = get_sample_url(1_000_000)
async with AsyncClient() as client:
    img_rec = await fetch(client, url)  # console blocks here
print(f'{img_rec.size:12_} bytes | {img_rec.name}')
Image(value=img_rec.pixels)

     982_309 bytes | Maurycy_Gottlieb_-_Jews_Praying_in_the_Synagogue_on_Yom_Kippur.jpg


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xdb\x00C\x00\x02\x01\x01\x01\…

Mais um trecho de [wikipics-demo](wikipics-demo.ipynb), para comparação:

```python
img_widgets = []
qty = 10
urls = get_sample_urls(2_000_000, qty)
total_bytes = 0

for url in urls:
    img_rec = fetch(url)
    total_bytes += img_rec.size
    display(Image(value=img_rec.pixels, layout=Layout(width='20%')))
    print(f'{img_rec.size:12_} bytes | {img_rec.name}')

print(f'TOTAL BYTES: {total_bytes:_}')
```


In [5]:
import asyncio

img_widgets = []
qty = 10
urls = get_sample_urls(2_000_000, qty)
total_bytes = 0

async def download_pics(urls):
    async with AsyncClient() as client:
        tasks = [fetch(client, url) for url in urls]
        results = await asyncio.gather(*tasks)
    return results

for img_rec in await download_pics(urls):
    total_bytes += img_rec.size
    display(Image(value=img_rec.pixels, layout=Layout(width='20%')))
    print(f'{img_rec.size:12_} bytes | {img_rec.name}')

print(f'TOTAL BYTES: {total_bytes:_}')    

Image(value=b'\xff\xd8\xff\xe1$\tExif\x00\x00II*\x00\x08\x00\x00\x00\x0c\x00\x0e\x01\x02\x00\xbb\x00\x00\x00\x…

   1_972_894 bytes | Barack_Obama_family_portrait_2011.jpg


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x01\x00H\x00H\x00\x00\xff\xe1$\x1fExif\x00\x00MM\x00*\x…

   1_964_661 bytes | Indischer_Maler_des_6._Jahrhunderts_001.jpg


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00\x00d\x00d\x00\x00\xff\xec\x00\x11Ducky\x00\x01\x00\…

...

   2_010_325 bytes | Rana_esculenta_on_Nymphaea_edit.JPG


Image(value=b'\xff\xd8\xff\xe1\x16\x14Exif\x00\x00II*\x00\x08\x00\x00\x00\n\x00\x0f\x01\x02\x00\x06\x00\x00\x0…

   2_015_977 bytes | Litoria_infrafrenata_-_Julatten.jpg


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x02\x00&\x00&\x00\x00\xff\xdb\x00C\x00\x02\x01\x01\x01\…

   1_977_558 bytes | Frederic_Edwin_Church_-_Aurora_Borealis_-_Google_Art_Project.jpg
TOTAL BYTES: 19_980_607


In [10]:
# %%time
# Não funciona:
#   File <timed exec>:18
# SyntaxError: 'await' outside function

from gallery import Gallery
from httpx import HTTPStatusError

num_images = 10

urls = get_sample_urls(15_000_000, num_images)
gallery = Gallery(num_images)
gallery.display()

async def display_pics(urls):
    async with AsyncClient() as client:
        tasks = [fetch(client, url) for url in urls]
        for i, coroutine in enumerate(asyncio.as_completed(tasks)):
            try:
                img_rec = await coroutine
            except HTTPStatusError:
                pass
            else:
                gallery.update(i, img_rec.pixels, img_rec.name)

await display_pics(urls)

print(f'TOTAL BYTES: {gallery.size:_}')

( 1)  15_283_554 bytes | Chehel_Sotoun_Inside%2C_Isfahan_Edit1.jpg
( 2)  15_193_424 bytes | Lady_Elliot_Island_SVII.jpg
( 3)  15_508_943 bytes | Sg%C3%B9rr_nan_Gillean_from_Sligachan%2C_Isle_of_Skye%2C_Scotland_-_Diliff.jpg
( 4)  14_784_640 bytes | Zaandam2.jpg
( 5)  14_726_913 bytes | Peacock_Flounder_Bothus_mancus_in_Kona.jpg
( 6)  15_062_885 bytes | Pahalgam_Valley.jpg
( 7)  15_217_910 bytes | USA_declaration_independence.jpg
( 8)  15_102_249 bytes | Hubble2005-01-barred-spiral-galaxy-NGC1300.jpg
( 9)  14_975_113 bytes | Shoki2_detail.jpg
(10)  15_680_406 bytes | Blois_Loire_Panorama_-_July_2011.jpg
TOTAL BYTES: 151_536_037
